In [5]:
%%capture
%run 01_hash.ipynb
%run 02_transaction.ipynb

In [6]:
import subprocess as sub

## 99_contract

Our goal is to be able to run a small piece of code in a tx, just like Ethereum. This piece of code is also called a smart contract (sc). The sc is like another user, but in addition to the balance it also has storage for variables and the sc itself. 

The smart contract should be able to access the current tx and block information.

## TODO:

- Inject block info

The smart contract that we are going to run contains the following code:

In [76]:
code = """
def init():
    r = 0
    s = 2
    h = 'hello how are you?'
    l = ['this', 'is', 'a', 'list']

r += 5
time = _tx['time'] + 'XXXXX'
l.append('XXXXX')
"""

Temporary file name before execution starts.

In [77]:
F = 'temp.py'

This is injected into every execution to get all the variables used in the contract.

In [78]:
PRINT_VARS = ['local = set(locals())',
              'local_vars = {}',
              'for n in local:',
              '    if not n.startswith(\'_\'):',
              '         if \'str\' in str(type(eval(n))): print(n, f\'\"{eval(n)}\"\', \'str\')',
              '         else: print(n, eval(n), \'int\')']

In [79]:
PRINT_VARS = """
\n
import json
_types = [str, float, int, list]                                                                         
out = {}                                                                                                 
keys = list(locals().keys())                                                                             
for k in keys:                                                                                           
    if not k.startswith('_'):                                                                            
        v = locals()[k]                                                                                  
        for t in _types:                                                                                 
            if isinstance(v, t): out[k] = [v, str(t)]                                                    
print(json.dumps(out)) 
"""
PRINT_VARS = PRINT_VARS.rstrip()

All vars used will be stored in the `STORAGE` dict.

In [80]:
STORAGE = {}

Inject transaction `tx` and `block` into the execution. These can be accessed in the sc.

In [81]:
def inject(tx, code, block=None): 
    tx = tx.json().replace('true', 'True')
    return '_tx = '+tx+'\n'+code

In [82]:
def save(code):
    with open(F, 'w+') as f: f.write(code)

In [83]:
def run():
    r = sub.check_output(f'python3 {F}', shell=True)
    return json.loads(r)

Stores all variables in `STORAGE`.

In [84]:
def store(r):
    for k,v in r.items(): STORAGE[k] = [v[0],v[1]]

The deployment will run all the code in `init()`. The rest will run in `execution`.

In [85]:
def deployable(code):
    init = []
    lines = code.split('\n')
    for i,l in enumerate(lines):
        if 'init' in l: 
            for j in range(i+1, len(lines)):
                if lines[j][:1] == ' ': init.append(lines[j].strip())
                else                  : break
    init.append(PRINT_VARS)
    return '\n'.join(l for l in init)

In [86]:
def deploy(code, tx):
    dc = deployable(code)
    ic = inject(tx, dc)
    save(ic)
    r  = run()
    store(r)

deploy(code, tx1); STORAGE

{'r': [0, "<class 'int'>"],
 's': [2, "<class 'int'>"],
 'h': ['hello how are you?', "<class 'str'>"],
 'l': [['this', 'is', 'a', 'list'], "<class 'list'>"]}

Execution will run everything in the sc except what is inside `init()`.

In [87]:
def executable(code):
    exe = []
    for k,v in STORAGE.items():
        if 'str' in v[1]: exe.append(f'{k} = "{v[0]}"')
        else            : exe.append(f'{k} = {v[0]}')
    lines = code.split('\n')
    in_init = False
    for l in lines:
        if 'init' in l: in_init = True; continue
        if in_init:
            if l[:1] == ' ': continue
            else           : in_init = False
        else: exe.append(l)
    exe.append(PRINT_VARS)
    return '\n'.join(l for l in exe)

In [88]:
def exe(code, tx):
    ic  = inject(tx, code)
    exe = executable(ic)
    save(exe)
    r  = run()
    store(r)
    
exe(code, tx1); STORAGE

{'r': [5, "<class 'int'>"],
 's': [2, "<class 'int'>"],
 'h': ['hello how are you?', "<class 'str'>"],
 'l': [['this', 'is', 'a', 'list', 'XXXXX'], "<class 'list'>"],
 'time': ['Mon Apr 12 11:41:43 2021XXXXX', "<class 'str'>"]}